# Получение данных

In [1]:
!git clone https://github.com/Derinhelm/llm_tokenization

Cloning into 'llm_tokenization'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 19 (delta 8), reused 13 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (19/19), 22.39 MiB | 5.98 MiB/s, done.
Resolving deltas: 100% (8/8), done.
Updating files: 100% (11/11), done.


In [2]:
import sys
sys.path.append('/content/llm_tokenization')

In [3]:
from data_classes import ConllEntry, Sentence

In [4]:
import pickle

with open('/content/llm_tokenization/treebank_test_sets/treebank_data.pickle', 'rb') as f:
    treebanks = pickle.load(f)

In [5]:
treebank_names = list(treebanks.keys())
treebank_names

['gsd', 'pud', 'syntagrus', 'poetry', 'taiga']

In [6]:
token_texts = {t.form for tr in treebank_names for s in treebanks[tr] for t in s.tokens if '.' not in t.id}

In [7]:
len(token_texts)

47088

# Создание набора подтокенов

In [8]:
from transformers import AutoTokenizer

model_names = ["Qwen/Qwen3-4B", "RefalMachine/RuadaptQwen3-4B-Instruct"]


In [9]:
tokenizers = {}
for m_name in model_names:
    m_title = m_name.split("/")[1].replace("-", "_")
    tokenizers[m_title] = AutoTokenizer.from_pretrained(m_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/12.4M [00:00<?, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/759 [00:00<?, ?B/s]

In [10]:
model_titles = list(tokenizers.keys())
model_titles

['Qwen3_4B', 'RuadaptQwen3_4B_Instruct']

In [13]:
def create_inputs(tokenizer_param, text):
    messages = [{"role": "user", "content": f"{text}"}]
    text = tokenizer_param.apply_chat_template(
            messages, tokenize=False,
            add_generation_prompt=False)
    inputs = tokenizer_param(text, return_offsets_mapping=True, return_tensors='pt')
    subtoken_texts = [tokenizer_param.decode(t) for t in inputs['input_ids'][0]]

    big_subtokens = []
    amount = 1
    id_list = inputs['input_ids'][0]
    while len(id_list) != 0:
      s1 = tokenizer_param.decode(id_list[:amount])
      if any([ord(symb) == 65533 for symb in s1]):
        amount += 1
      else:
        big_subtokens.append((s1, amount))
        id_list = id_list[amount:]
        amount = 1
    return (inputs, subtoken_texts, big_subtokens)

In [14]:
sent_subtokens = {}
for m_title in model_titles:
  sent_subtokens[m_title] = {}
  for tr in treebank_names:
    sent_subtokens[m_title][tr] = {}
    for sent in treebanks[tr]:
      sent_subtokens[m_title][tr][sent.sent_id] = create_inputs(tokenizers[m_title], sent.text)

In [15]:
with open(f'/content/sent_subtokens.pickle', 'wb') as f:
    pickle.dump(sent_subtokens, f)

In [16]:
from google.colab import files
files.download('sent_subtokens.pickle')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>